In [1]:
from bs4 import BeautifulSoup
import numpy as np
import requests

In [2]:
request = requests.get("https://www.biznesradar.pl/wskazniki-wartosci-rynkowej/GIG", allow_redirects=False)
html_code = BeautifulSoup(request.content, 'html.parser')


In [165]:
class CompanyIndexScraper:
    SKELETON_URL = "https://www.biznesradar.pl"
    INDEXES_TO_SCRAP = {
        "wskazniki-wartosci-rynkowej" : [
            "Cena / Wartość księgowa",
            "Cena / Wartość księgowa Grahama",
            "Cena / Przychody ze sprzedaży",
            "Cena / Zysk",
            "Cena / Zysk operacyjny",
            "EV / Przychody ze sprzedaży",
            "EV / EBIT",
            "EV / EBITDA",
        ],
        "wskazniki-rentownosci" : [
            "ROE",
            "ROA",
            "Marża zysku operacyjnego",
            "Marża zysku netto",
            "Marża zysku ze sprzedaży",
            "Marża zysku brutto",
            "Marża zysku brutto ze sprzedaży",
            "Rentowność operacyjna aktywów",
        ],
        "wskazniki-przeplywow-pienieznych" : [
            "Udział zysku netto w przepływach operacyjnych",
            "Wskaźnik źródeł finansowania inwestycji",
        ],
        "wskazniki-zadluzenia" : [
            "Zadłużenie ogólne",
            "Zadłużenie kapitału własnego",
            "Zastosowanie kapitału obcego",
            "Wskaźnik ogólnej sytuacji finansowej",
        ],
        "wskazniki-plynnosci" : [
            "I stopień pokrycia",
            "Płynność gotówkowa",
            "Płynność szybka",
            "Płynność bieżąca",
            "Pokrycie zobowiązań należnościami",
        ],
    }

    DATA_TYPE = [
        "Company",
        "Sector",
        "CompanyY/Y",
        "SectorY/Y",
        "CompanyQ/Q",
        "SectorQ/Q",
    ]

    def __init__(self):
        self.dates_header_ = None
        self.html_table_   = None
        self.data_array_   = None
        self.columns_name_ = sum(CompanyIndexScraper.INDEXES_TO_SCRAP.values(), [])


    def __generate_url_address(self, indexes_type : str, ticker : str) -> str:
        return f"{CompanyIndexScraper.SKELETON_URL}/{indexes_type}/{ticker}"
    

    def __download_http_response(self, url : str) -> BeautifulSoup:
        request = requests.get(url, allow_redirects=False)
        # If ticker is different or if it is shortcuts
        if request.status_code == 301:
            url = f"{CompanyIndexScraper.SKELETON_URL}{request.headers['Location']}"
            request = requests.get(url)
        # If some error with request raise exception
        if request.status_code != 200:
            raise Exception(f"Download http response error code: {request.status_code} on address {url}")
        return BeautifulSoup(request.content, 'html.parser')


    def __download_and_prepare_html_table(self, indexes_key : str, ticker : str) -> None:
        url = self.__generate_url_address(indexes_key, ticker)
        html_response = self.__download_http_response(url)
        self.html_table_ = html_response.find("table", attrs={"class":"report-table"}) 


    def __scrap_header_date(self):
        header_row = self.html_table_.find("tr")
        self.dates_header_ = {"Date": ["".join(date.text.split()).split('(')[0] for date in header_row.find_all("th", attrs={"class":"thq h"})]}


    def __prepare_data_space(self) -> None:
        # Before use dates_header_ should be initialized 
        self.data_array_ = np.empty((len(self.columns_name_), len(self.dates_header_["Date"]), len(CompanyIndexScraper.DATA_TYPE)))
        self.data_array_[:] = np.nan
    

    def __scrap_data_table(self, index_key : str) -> None:
        data_rows = self.html_table_.find_all("tr")[1:] # Omit header line
        for row in data_rows:
            self.__scrap_data_row(index_key, row) 


    def __scrap_data_row(self, index_key : str, row : BeautifulSoup) -> None:
        # Check is row to scrap 
        row_header = row.find("td", attrs={"class":"f"}).text
        # If row not in INDEXES_TO_SCRAP omit row
        if row_header not in CompanyIndexScraper.INDEXES_TO_SCRAP[index_key]:
            return
        
        cells = row.find_all("td", attrs={"class":"h"})[:-1]
        n_row = self.columns_name_.index(row_header)
        for n_col, cell in enumerate(cells):
            self.__scrap_data_cell(cell, n_row, n_col) 
    
    
    def __scrap_data_cell(self, cell : BeautifulSoup, n_row : int, n_col : int) -> None:
        # Index company value
        raw_section = cell.find("span", attrs={"class": "value"})
        if raw_section:
            self.data_array_[n_row, n_col, 0] = float(raw_section.find("span", attrs={"class":"pv"})
                                                      .text.replace("%", "").replace(" ", ""))
        
        # Index sector value
        raw_section = cell.find("span", attrs={"class": "sectorv"})
        if raw_section:
            self.data_array_[n_row, n_col, 1] = float(raw_section.find("span", attrs={"class":"pv"})
                                                      .text.replace("%", "").replace(" ", ""))
        
        # Change Q/Q
        raw_section = cell.find("div", attrs={"class": "changeqq"})
        if raw_section:
            self.data_array_[n_row, n_col, 2] = float(raw_section.find("span", attrs={"class":"pv"})
                                                      .text.replace("%", "").replace(" ", ""))
            self.data_array_[n_row, n_col, 3] = float(raw_section.find("span", attrs={"class":"sectorv"})
                                                    .find("span", attrs={"class":"pv"}).text.replace("%", "").replace(" ", ""))
            
        # Change Y/Y
        raw_section = cell.find("div", attrs={"class": "changeyy"})
        if raw_section:
            self.data_array_[n_row, n_col, 4] = float(raw_section.find("span", attrs={"class":"pv"})
                                                      .text.replace("%", "").replace(" ", ""))
            self.data_array_[n_row, n_col, 5] = float(raw_section.find("span", attrs={"class":"sectorv"})
                                                  .find("span", attrs={"class":"pv"}).text.replace("%", "").replace(" ", ""))


    def scrap_data(self, ticker : str):
        for index_key in CompanyIndexScraper.INDEXES_TO_SCRAP.keys():
            self.__download_and_prepare_html_table(index_key, ticker)
            self.__scrap_header_date()
            self.__prepare_data_space()
            self.__scrap_data_table(index_key)
            



In [166]:
cis = CompanyIndexScraper()
cis.scrap_data("ATR")

In [170]:
cis.data_array_[:,:,2]

array([[   nan,    nan,    nan, ...,    nan,    nan,    nan],
       [   nan,    nan,    nan, ...,    nan,    nan,    nan],
       [   nan,    nan,    nan, ...,    nan,    nan,    nan],
       ...,
       [   nan,    nan,  12.88, ...,  -3.78,   3.67,   0.66],
       [   nan,    nan,  15.93, ...,  -3.87,   3.62,   0.65],
       [   nan,    nan,  25.44, ...,   2.08,  -3.75, -25.39]])

In [172]:
import pandas as pd

pd.DataFrame(cis.data_array_[:,:,0])

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
